In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.interpolate as si
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, RANSACRegressor,\
                                 TheilSenRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error

In [2]:
def get_bspline_basis(knots, degree=3, periodic=False):
    """Get spline coefficients for each basis spline."""
    nknots = len(knots)
    y_dummy = np.zeros(nknots)

    knots, coeffs, degree = si.splrep(knots, y_dummy, k=degree,
                                      per=periodic)
    ncoeffs = len(coeffs)
    bsplines = []
    for ispline in range(nknots):
        coeffs = [1.0 if ispl == ispline else 0.0 for ispl in range(ncoeffs)]
        bsplines.append((knots, coeffs, degree))
    return bsplines

In [3]:
class BSplineFeatures(TransformerMixin):
    def __init__(self, knots, degree=3, periodic=False):
        self.bsplines = get_bspline_basis(knots, degree, periodic=periodic)
        self.nsplines = len(self.bsplines)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        nsamples, nfeatures = X.shape
        features = np.zeros((nsamples, nfeatures * self.nsplines))
        for ispline, spline in enumerate(self.bsplines):
            istart = ispline * nfeatures
            iend = (ispline + 1) * nfeatures
            features[:, istart:iend] = si.splev(X, spline)
        return features

In [4]:
nyc_pumpkins = pd.read_csv("./new-york_9-24-2016_9-30-2017.csv")
cat_map = {
    'sml': 0,
    'med': 1,
    'med-lge': 2,
    'lge': 3,
    'xlge': 4,
    'exjbo': 5
}
nyc_pumpkins = nyc_pumpkins.assign(
    size=nyc_pumpkins['Item Size'].map(cat_map),
    price=nyc_pumpkins['High Price'] + nyc_pumpkins['Low Price'] / 2,
    size_class=(nyc_pumpkins['Item Size'].map(cat_map) >= 2).astype(int)
)
nyc_pumpkins = nyc_pumpkins.drop([c for c in nyc_pumpkins.columns if c not in ['size', 'price', 'size_class']], 
                                 axis='columns')
nyc_pumpkins = nyc_pumpkins.dropna()

In [5]:
nyc_pumpkins.head(10)

,size,price,size_class
0,4.0,245.0,1
1,3.0,245.0,1
2,4.0,215.0,1
3,3.0,215.0,1
4,2.0,200.0,1
5,4.0,245.0,1
6,3.0,230.0,1
7,1.0,280.0,0
8,4.0,245.0,1
9,3.0,245.0,1


In [6]:
def aic(y, y_pred, k):
    resid = y - y_pred.ravel()
    sse = sum(resid ** 2)
    AIC = 2*k - 2*np.log(sse)

    return -AIC

In [7]:
X_train, X_test, y_train, y_test = train_test_split(nyc_pumpkins['price'], nyc_pumpkins['size'], test_size=0.20, random_state=42, shuffle=True) #split 20% into test set

In [8]:
def get_r2_and_aic_for_both_models(X_train, y_train, X_test, y_test):
    polynomial_features = PolynomialFeatures(degree=2,
                                             include_bias=True)
    linear_regression = LinearRegression()
    model_polynomial = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    model_polynomial.fit(X_train[:, np.newaxis], y_train)

    predicted_sizes_poly = np.round(model_polynomial.predict(X_test[:, np.newaxis]))
    
    X_train = np.array(X_train).reshape(-1,1)
    X_test = np.array(X_test).reshape(-1,1)

    y_train = np.array(y_train).reshape(-1,1)
    y_test = np.array(y_test).reshape(-1,1)
    
    knots = np.linspace(0, 500, 10)
    bspline_features = BSplineFeatures(knots, degree=2, periodic=False)
    model_splines = make_pipeline(bspline_features, LinearRegression(fit_intercept=False))
    model_splines.fit(X_train, y_train)    
    # Evaluate the models using crossvalidation
    predicted_sizes_splines = np.round(model_splines.predict(X_test))
    
    aic_poly = aic(y_test.ravel(), predicted_sizes_poly, k=2)
    aic_splines = aic(y_test.ravel(), predicted_sizes_splines, k=2)
    r2_poly = r2_score(y_test.ravel(), predicted_sizes_poly)
    r2_splines = r2_score(y_test.ravel(), predicted_sizes_splines)
    return aic_poly, aic_splines, r2_poly, r2_splines

In [9]:
aic_poly, aic_splines, r2_poly, r2_splines = get_r2_and_aic_for_both_models(X_train, y_train, X_test, y_test)

In [10]:
X_train[20]= -10
y_train[20] = 30

In [11]:
aic_poly_outlier, aic_splines_outlier, r2_poly_outlier, r2_splines_outlier = get_r2_and_aic_for_both_models(X_train, y_train, X_test, y_test)

In [12]:
print('AIC for Poly model was {}'.format(aic_poly))
print('AIC for Splines model was {}'.format(aic_splines))
print('R2 for Poly model was {}'.format(r2_poly))
print('R2 for Splines model was {}'.format(r2_splines))

AIC for Poly model was 1.2781146592305168
AIC for Splines model was 2.7345916599729483
R2 for Poly model was 0.7554076539101497
R2 for Splines model was 0.4933444259567389


In [13]:
print('AIC for Poly model with outlier was {}'.format(aic_poly_outlier))
print('AIC for Splines model with outlier was {}'.format(aic_splines_outlier))
print('R2 for Poly model with outlier was {}'.format(r2_poly_outlier))
print('R2 for Splines model with outlier was {}'.format(r2_splines_outlier))


AIC for Poly model with outlier was 3.2218358252884487
AIC for Splines model with outlier was 2.802394763324311
R2 for Poly model with outlier was 0.35357737104825304
R2 for Splines model with outlier was 0.47587354409317817
